<a href="https://colab.research.google.com/github/Chalita378/FaceDetection/blob/main/ResNet50_train%2Bvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference : https://medium.com/analytics-vidhya/implementing-cnn-in-pytorch-with-custom-dataset-and-transfer-learning-1864daac14cc


# Mount drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My Drive/Dataset/./tarball-lite/AFAD-Lite/

/content/drive/My Drive/Dataset/tarball-lite/AFAD-Lite


In [ ]:
# !git clone https://github.com/afad-dataset/tarball-lite.git

In [ ]:
# %cd ./tarball-lite/AFAD-Lite/

In [ ]:
# !cat AFAD-Lite.tar.xz* > AFAD-Lite.tar.xz

In [ ]:
# !tar -xf AFAD-Lite.tar.xz

# เรียก dataset เป็น csv ที่เตรียมไว้


In [ ]:
import pandas as pd
import os
import torch

device = ("cuda" if torch.cuda.is_available () else "cpu")

df = pd.read_csv('train_df.csv')

In [ ]:
df.Age.value_counts()

In [ ]:
df.shape

In [ ]:
print(device)

# Creating Custom Dataset


In [ ]:
from torch.utils.data.dataset import Dataset
import pandas as pd
import os
from PIL import Image
import torch
from skimage import io

class FaceDataset(Dataset):
  def __init__(self, root_dir, annotation_file, transform=None):
    self.root_dir = root_dir
    self.annotation = pd.read_csv(annotation_file)
    self.transform = transform

  def __len__(self):
    return len(self.annotation)

  def __getitem__(self, index):
    img_id = self.annotation.iloc[index, 1]
    img = Image.open(os.path.join(self.root_dir, str(img_id))).convert("RGB")
    y_label = self.annotation.iloc[index, 2:]
    y_label = y_label.astype("float", copy=False)
    # y_label = pd.to_numeric(torch.tensor(self.annotation.iloc[0, 2:]), errors='coerce')
    y_label = torch.tensor(y_label)
    if self.transform:
      img = self.transform(img)
    
    return (img, y_label)

# Creating Model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CNN(nn.Module):
  def __init__(self, train_CNN=False, num_classes=22):
      super(CNN, self).__init__()
      self.train_CNN = train_CNN
      self.resnet50 = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
      self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, num_classes)
      self.relu = nn.ReLU()
      self.dropout = nn.Dropout(0.5)
      self.linear = nn.Sequential()
      # self.linear2 = nn.Linear(2048, 22)
      self.resnet50.eval()

  def forward(self, images):
    features = self.resnet50(images)

    return features
    return self.linear(self.dropout(self.relu(features))).squeeze(1)

# Check Layers of network. 


# Importing Libraries for training loop

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
import numpy as np 

device = ("cuda" if torch.cuda.is_available() else "cpu")

# Transformations

In [ ]:
transform = transforms.Compose([
        transforms.Resize((356, 356)),
        transforms.RandomCrop((299,299)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

### Hyperparameters

In [ ]:
num_epochs = 20
learning_rate = 0.001
train_LeNet = False
batch_size = 32
shuffle = True
pin_memory = True
num_workers = 2

In [ ]:
dataset = FaceDataset("train","train_dataset.csv",transform=transform)
# lengths = [round(int(len(dataset)*0.7)), round(int(len(dataset)*0.3))]
train_set, validation_set, test_set = torch.utils.data.random_split(dataset,[40000, 10000, 9344])
train_loader = DataLoader(dataset=train_set, shuffle=shuffle,
                          batch_size=batch_size,num_workers=num_workers,
                          pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle,
                               batch_size=batch_size,num_workers=num_workers,
                               pin_memory=pin_memory)
test_loader = DataLoader(dataset=test_set, shuffle=False,
                               batch_size=batch_size,num_workers=num_workers,
                               pin_memory=pin_memory)

In [ ]:
# class WeightedFocalLoss(nn.Module):
#     "Non weighted version of Focal Loss"
#     def __init__(self, alpha=.25, gamma=2):
#         super(WeightedFocalLoss, self).__init__()
#         self.alpha = torch.tensor([alpha, 1-alpha]).cuda()
#         self.gamma = gamma

#     def forward(self, inputs, targets):
#         BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
#         targets = targets.type(torch.long)
#         at = self.alpha.gather(0, targets.data.view(-1))
#         pt = torch.exp(-BCE_loss)
#         F_loss = at*(1-pt)**self.gamma * BCE_loss
#         return F_loss.mean()

In [ ]:
model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size = 5, gamma = 0.1)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
def check_accuracy(loader, model):
    if loader == str(train_loader):
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on validation data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            predictions = torch.tensor([1.0 if i >= 0.5 else 0.0 for i in scores]).to(device)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    return f"{float(num_correct)/float(num_samples)*100:.2f}"

    print(
        f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
    model.train()

In [ ]:
def train():
    model.train()
    for epoch in range(num_epochs):
      training_loss = 0.0
      val_loss = 0.0
      val_acc = 0
      correct_preds = 0
      correct_train = 0
      best_acc = 0
      validation = 0.0
      total = 0
      total_train_acc = 0

      loop = tqdm(enumerate(train_loader), total = len(train_loader), leave=False)
      
      for batch_idx, (x_imgs, y_labels) in loop:
        x_imgs = x_imgs.to(device)
        y_labels = y_labels.to(device)
        outputs = model(x_imgs)

        _, predicted = torch.max(outputs, 0)
        _, lalabel = torch.max(y_labels.data, 0)
        correct_train += (predicted == lalabel).sum().item()
        total_train_acc += y_labels.size(0)
        
        loss = criterion(outputs.float(), y_labels.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        training_loss += loss.item()

      valid_data_loader = tqdm(validation_loader, total=len(validation_loader), leave=True)

      with torch.no_grad():
        model.eval()
        for x_imgs, y_labels in valid_data_loader:
            x_imgs = x_imgs.to(device)
            y_labels = y_labels.to(device)
            outputs = model(x_imgs)

            val_loss = criterion(outputs.float(), y_labels.float())
            validation += val_loss.item()

            total += y_labels.size(0)
            _, predicted = torch.max(outputs.data, 0)
            _, lalabel = torch.max(y_labels.data, 0)
            correct_preds += (predicted == lalabel).sum().item()

        train_acc = 100 * (correct_train / total_train_acc)
        training_loss /= len(loop)
        validation /= len(valid_data_loader)
        val_acc = 100* (correct_preds / total)

        print(f'Epoch: {epoch+1}/{num_epochs}.. Training loss: {training_loss}.. Validation Loss: {validation}')
        print('Training Accuracy is: {:.2f}%'.format(train_acc))
        print('Validation Accuracy is: {:.2f}%'.format(val_acc))

if __name__ == "__main__":
    train()


In [ ]:
print(prediction[1].cpu().numpy())
print(y_labels[1].cpu().numpy())
print(np.sum(prediction[1].cpu().numpy()==y_labels[1].cpu().numpy()))


[ 0  0  0  4  0  0 13  0  5  5  0  0  0  0  0  0  0  0 14  0  0  0  0  0
  0  0  4  4  0  0  0  0]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
PATH = "ResNet_20epoch_.pth"
torch.save(model, PATH)

In [ ]:
load_model = torch.load("ResNet_10epoch_.pth")
load_model.eval()
# print(load_model)

CNN(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    batch_size,
    epochs,
    validation_data=(x_val, y_val),
)

In [ ]:
!git clone http://github.com/icpm/pytorch-cifar10.git
%cd pytorch-cifar10/

fatal: destination path 'pytorch-cifar10' already exists and is not an empty directory.
/content/drive/My Drive/Dataset/tarball-lite/AFAD-Lite/pytorch-cifar10


In [ ]:
import numpy as np 
from misc import progress_bar

test_loss = 0.0
total     = 0.0
test_correct =0.0

for i, (x_imgs, y_labels) in enumerate(test_loader, 0):
  ### Extract image and labels from trainloader
  x_imgs  = x_imgs.to(device)
  y_labels  = y_labels.to(device)

  outputs = load_model(x_imgs)

  loss    = criterion(outputs, y_labels)
  test_loss += loss.item()

  ### select class index with the maximum value 
  prediction = torch.max(outputs, 0)
  total     += y_labels.size(0)
  test_correct += np.sum(prediction[1].cpu().numpy()==y_labels.cpu().numpy())
  progress_bar(i, len(test_loader), 'Loss: %.4f | Acc: %.3f%% (%d/%d)'
                             % (test_loss / (i + 1), 100. * test_correct / total, test_correct, total))


print('test_loss = '+str(test_loss))
print('test_accuracy = '+ str(test_correct/total))

 292/292 [================================================================================>]  Step: 7s527ms | Tot: 20m5s | Loss: 0.0424 | Acc: 86.376% (8071/9344)
test_loss = 12.392186460909226
test_accuracy = 0.8637628424657534
